<a href="https://colab.research.google.com/github/vinicius-vargas/robust-market-screener/blob/main/stock_valuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pandas-datareader yahooquery investpy fundamentus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.3 MB/s eta 0:00:00


In [ ]:
### Setting up libraries
from google.colab import drive
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import time
from pandas_datareader import data as pdr
import yahooquery as yq
import investpy as inv
import fundamentus as fd
import warnings

warnings.filterwarnings("ignore")

# Connect to Google Drive (My data lake)
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
### Read data from my data lake
df=pd.read_csv('/content/drive/My Drive/data_lake/alpha_raking.csv')

# Get current date t
curr_date = datetime.today().strftime('%Y-%m-%d')

# Check if the Last Update is equal today
df = df[(df.last_update == curr_date)]

df.head(3)

,ticker,qtd_dias,s_p,bond_emg,cmmdt,usd_real,mkt,size,qld,momnt,liq,return,vol,alpha,r_score,last_update
0,PRIO3.SA,2345,0.208,0.234,0.545,0.081,0.683,0.082,0.072,0.048,0.092,5.233,0.04098,0.00153,0.273,2024-12-17
1,UNIP6.SA,2345,0.168,0.085,0.112,-0.015,0.463,0.134,0.039,0.073,0.127,3.941,0.02631,0.00124,0.209,2024-12-17
2,DEXP3.SA,2345,0.161,0.124,0.085,-0.083,0.367,0.257,0.012,-0.044,0.069,3.018,0.04682,0.00102,0.056,2024-12-17


In [ ]:
def tidy_fin(ticker, data, freq, hist_prices):
  # Get sector
  sector = data.asset_profile[ticker]['industry']

  # Load do balance sheet
  asst_annual_fin = (
      data
      .get_financial_data(
          types = ['TotalRevenue', 'GrossProfit', 'NetIncome', 'CommonStockEquity', 'ShareIssued'],
          frequency = freq
      )
      .query("periodType != 'TTM'")
  )

  # Check if GrossProfit is in DF
  if 'GrossProfit' not in asst_annual_fin.columns:
    asst_annual_fin['GrossProfit'] = asst_annual_fin['TotalRevenue']

  # Transform index date frequency
  asst_annual_fin.index = asst_annual_fin.asOfDate.apply(pd.to_datetime).dt.year

  # Left Join with Prices Table and Mutate it
  asst_annual_fin = (
      asst_annual_fin
      .merge(
          last_prices[last_prices.index.isin(asst_annual_fin.index)],
          left_index=True,
          right_index=True
      )
      .assign(
          ticker = ticker,
          sector = sector,
          gross_margin = asst_annual_fin['GrossProfit'] / asst_annual_fin['TotalRevenue'],
          net_margin = asst_annual_fin['NetIncome'] / asst_annual_fin['TotalRevenue'],
          psr = last_prices['last'] / (asst_annual_fin['TotalRevenue'] / asst_annual_fin['ShareIssued']),
          pvp = last_prices['last'] / (asst_annual_fin['CommonStockEquity'] / asst_annual_fin['ShareIssued']),
          pl = last_prices['last'] / (asst_annual_fin['NetIncome'] / asst_annual_fin['ShareIssued'])
      )
      [['ticker', 'sector', 'gross_margin',	'net_margin',	'psr',	'pvp',	'pl']]
      .sort_index(ascending=True)
  )

  return asst_annual_fin



# function to check if a value is '-'
def compare(x):
  if x=='-' or x=='':
    return 0.0
  else:
    return float(x)

In [ ]:
# Select tickers to get fundamentalist informations #'DEXP3.SA'
lista = df['ticker']

# Create dataframe to save fundamental indexes
data = pd.DataFrame()

for ticker in tqdm(lista):
  ### Get the Historical Company Performance - Gross and Net Margin
  #################################################################
  df_fd = fd.get_papel(ticker[:-3])

  # Get Historical Prices & Dividends
  yf_data = yq.Ticker(ticker)
  asst_data = yf_data.history(period = '10y').reset_index(0)
  asst_data['year'] = pd.to_datetime(asst_data.index.to_series(), errors='coerce', utc=True).dt.year

  if 'dividends' not in asst_data.columns:
    asst_data['dividends'] = 0

  ### Get the last price of each year
  time.sleep(0.2)
  last_prices = asst_data.groupby('year')['close'].agg(['last'])

  # Function do Get the Balance Sheet (or Balanco Patrimonial)
  hist_perf = (
      tidy_fin(
          ticker = ticker,
          data = yf_data,
          freq = 'a',
          hist_prices = last_prices
      )
      .groupby(['ticker', 'sector'])
      .median()
  )

  ### Get the Historical Dividend Yield
  #####################################

  ### Calculate the Dividend payed by year
  sun_div = asst_data[asst_data.dividends != 0].groupby('year')['dividends'].agg(['sum'])

  ### Grouping Last Price with Dividends Sum
  asst_div_data = pd.concat([last_prices, sun_div], axis=1)

  asst_div_data['yield'] = asst_div_data['sum'] / asst_div_data['last']

  hist_div = round(asst_div_data['yield'].median() * 100, 2)


  ### Final Dataset - Fundamentalist Performance & Index
  ######################################################

  final_data_fund = pd.DataFrame(
      {
      'ticker': ticker,
      'Setor': hist_perf.index[0][1],
      'Mg. Bruta Med.': [round(hist_perf['gross_margin'][0] * 100, 2)],
      'Mg. Bruta Hj': [compare(df_fd.Marg_Bruta[0][:-1])],
      'Mg. Net Med': [round(hist_perf['net_margin'][0] * 100, 2)],
      'Mg. Net Hj':[compare(df_fd.Marg_Liquida[0][:-1])],
      'ROIC': [compare(df_fd['ROIC'][0][:-1])],
      'ROE': [compare(df_fd['ROE'][0][:-1])],
      'Div. Liq / Patr': [compare(df_fd['Div_Br_Patrim'][0][:-1]) / 10],
      'Liq Corrente': [compare(df_fd['Liquidez_Corr'][0]) / 100],
      'Div. Yield Med': [hist_div],
      'Div. Yield Hj': [compare(df_fd.Div_Yield[0][:-1])],
      'PSR Med': [round(hist_perf['psr'][0], 2)],
      'PSR Hj':[compare(df_fd.PSR[0]) / 100],
      'PVP Med':[round(hist_perf['pvp'][0], 2)],
      'PVP Hj':[compare(df_fd.PVP[0]) / 100],
      'PL Med':[round(hist_perf['pl'][0], 2)],
      'PL Hj':[compare(df_fd.PL[0]) / 100]
      }
  )

  time.sleep(0.2)

  data = pd.concat([data, final_data_fund], ignore_index=True)

  0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
### SECTOR + INDUSTRY FILTERING & BLOCKLIST
###########################################

### BLOCKLIST - INDUSTRIES
blocklist_industry = [
    'Real Estate Services'
    ,'Real Estate—Development'
    ,'Real Estate - Development'
    ,'Residential Construction'
    ,'Airlines'
    ,'Specialty Retail'
    ,'Internet Retail'
    ,'Apparel Retail'
    ,'Packaged Foods'
    ,'Restaurants'
    ,'Oil & Gas Integrated'
    ,'Oil & Gas E&P'
    ,'Oil & Gas Refining & Marketing'
    ,'Oil & Gas Equipment & Services'
    ,'Steel'
]

### BLOCKLIST - COMPANIES
blocklist_companies = [
    'PETR4.SA'  ## Muita treta envolvida
    ,'VALE3.SA' ## Brumadinho e Mariana
    ,'AZUL4.SA' ## Setor ruim
    ,'GOLL4.SA' ## Setor ruim
    ,'JBSS3.SA'
    ,'AALR3.SA' ## TOP 1 Piores ESG
    ,'PNVL3.SA' ## TOP 2 Piores ESG
    ,'TRIS3.SA' ## TOP 3 Piores ESG
    ,'BRAP3.sa' ## TOP 4 Piores ESG
    ,'LAND3.SA' ## TOP 5 Piores ESG
]

### FILTER
data = data[
    (~data['Setor'].isin(blocklist_industry))
    & (~data['ticker'].isin(blocklist_companies))
].reset_index(drop=True)

data

,ticker,Setor,Mg. Bruta Med.,Mg. Bruta Hj,Mg. Net Med,Mg. Net Hj,ROIC,ROE,Div. Liq / Patr,Liq Corrente,Div. Yield Med,Div. Yield Hj,PSR Med,PSR Hj,PVP Med,PVP Hj,PL Med,PL Hj
0,UNIP6.SA,Chemicals,39.61,25.9,17.16,9.6,8.8,15.3,1.0,2.26,5.17,8.8,1.40,1.20,3.61,1.92,8.50,12.55
1,DEXP3.SA,Chemicals,20.21,19.7,10.59,10.3,15.0,17.2,0.2,3.61,3.03,7.2,0.59,0.51,1.86,0.92,5.45,5.39
2,STBP3.SA,Marine Shipping,43.46,56.3,19.47,27.4,27.0,33.0,1.1,3.36,4.04,20.2,3.70,4.06,2.86,4.90,15.88,14.84
3,SHUL4.SA,Auto Parts,24.46,25.8,13.20,14.0,15.6,19.4,0.4,3.18,5.26,7.8,1.01,1.02,1.64,1.42,8.15,7.32
4,BPAC11.SA,Capital Markets,100.00,0.0,34.03,0.0,0.0,20.2,0.0,0.00,1.99,1.9,13.81,0.00,7.63,1.99,40.82,9.84
5,SLCE3.SA,Farm Products,30.81,24.9,12.97,5.0,8.3,8.1,1.1,1.71,4.23,4.6,1.20,1.12,1.97,1.72,9.16,21.19
6,ROMI3.SA,Industrial Distribution,31.33,28.9,14.12,10.8,3.5,10.2,0.6,1.96,5.91,9.2,0.97,0.71,1.26,0.68,6.56,6.65
7,CSUD3.SA,Software - Infrastructure,34.90,41.4,12.73,16.7,22.2,20.1,0.1,1.23,5.09,6.1,1.22,1.12,1.75,1.35,9.14,6.73
8,FRAS3.SA,Auto Parts,29.26,31.6,9.66,9.3,15.8,15.7,0.5,2.26,3.14,2.9,1.25,1.59,2.41,2.79,12.42,17.77
9,CSMG3.SA,Utilities - Regulated Water,39.42,43.4,14.47,17.8,16.1,17.8,0.7,1.73,5.68,11.9,1.01,1.08,0.89,1.08,7.36,6.05


In [ ]:
######################################
### Filter All TOP Stocks - And Cheap!
######################################

final_data = data[

    (data['Mg. Bruta Med.'] >= 15)
    & ((data['Mg. Bruta Hj'] >= 15) | ((data['Setor'] == 'Intermediários Financeiros') & (data['Mg. Bruta Hj'] == 0)))

    & (data['Mg. Net Med'] > 5)
    & ((data['Mg. Net Hj'] > 5) | ((data['Setor'] == 'Intermediários Financeiros') & (data['Mg. Net Hj'] == 0)))

    & ((data['ROIC'] > 8) | ((data['Setor'] == 'Intermediários Financeiros') & (data['ROIC'] == 0)))
    & (data['ROE'] > 9)

    & (data['Div. Liq / Patr'] <= 2)
    & ((data['Liq Corrente'] >= 1) | ((data['Setor'] == 'Intermediários Financeiros') & (data['Liq Corrente'] == 0)))

    & (data['Div. Yield Med'] >= 3)
    & (data['Div. Yield Hj'] >= 3)

##    & (data['PSR Med'] <= 6)
    & (((data['PSR Hj'] < data['PSR Med'] * 0.85))

##    & (data['PVP Med'] <= 5)
    & (data['PVP Hj'] < data['PVP Med'] * 0.85))

    & (data['PL Med'] > 0)
##    & (data['PL Med'] <= 14)
    & (data['PL Hj'] > 0)
    & ((data['PL Hj'] <= 12) | (data['PL Hj'] < data['PL Med'] * 0.85))

  ].reset_index(drop=True)

final_data

,ticker,Setor,Mg. Bruta Med.,Mg. Bruta Hj,Mg. Net Med,Mg. Net Hj,ROIC,ROE,Div. Liq / Patr,Liq Corrente,Div. Yield Med,Div. Yield Hj,PSR Med,PSR Hj,PVP Med,PVP Hj,PL Med,PL Hj
0,TAEE11.SA,Utilities - Regulated Electric,75.76,62.2,59.46,36.4,11.7,20.6,1.4,1.42,10.01,10.4,11.30,2.97,5.67,1.68,20.81,8.16
1,ALUP11.SA,Utilities - Renewable,74.58,74.6,21.13,45.2,9.9,12.3,1.5,1.89,4.09,5.1,5.28,2.30,3.75,1.06,26.10,8.66
2,AGRO3.SA,Farm Products,44.72,34.9,30.17,23.9,8.9,13.0,0.5,2.01,5.98,6.7,2.31,1.94,1.30,1.05,9.73,8.13
3,B3SA3.SA,Financial Data & Stock Exchanges,79.76,90.0,48.87,41.5,18.8,22.5,0.6,1.86,3.20,4.1,9.04,5.28,4.01,2.87,19.47,12.73
4,FLRY3.SA,Diagnostics & Research,27.18,27.7,7.77,8.1,11.8,11.5,0.9,1.93,4.09,5.5,1.50,0.92,2.61,1.30,20.86,11.29


In [ ]:
####################################################
### Filter All TOP Stocks - But Expensive - TO WATCH
####################################################

to_watch = data[
    (data['Mg. Bruta Med.'] >= 20)
    & ((data['Mg. Bruta Hj'] >= 20) | ((data['Setor'] == 'Intermediários Financeiros') & (data['Mg. Bruta Hj'] == 0)))

    & (data['Mg. Net Med'] > 6)
    & ((data['Mg. Net Hj'] > 6) | ((data['Setor'] == 'Intermediários Financeiros') & (data['Mg. Net Hj'] == 0)))

    & ((data['ROIC'] > 8) | ((data['Setor'] == 'Intermediários Financeiros') & (data['ROIC'] == 0)))
    & (data['ROE'] > 9)

    & (data['Div. Liq / Patr'] <= 2)
    & ((data['Liq Corrente'] >= 1) | ((data['Setor'] == 'Intermediários Financeiros') & (data['Liq Corrente'] == 0)))

    & (data['Div. Yield Med'] >= 4)
    & (data['Div. Yield Hj'] >= 4)
    & (data['Div. Yield Hj'] < 20)

  ].reset_index(drop=True)

to_watch = to_watch[~to_watch.ticker.isin(final_data.ticker)]

to_watch

,ticker,Setor,Mg. Bruta Med.,Mg. Bruta Hj,Mg. Net Med,Mg. Net Hj,ROIC,ROE,Div. Liq / Patr,Liq Corrente,Div. Yield Med,Div. Yield Hj,PSR Med,PSR Hj,PVP Med,PVP Hj,PL Med,PL Hj
0,UNIP6.SA,Chemicals,39.61,25.9,17.16,9.6,8.8,15.3,1.0,2.26,5.17,8.8,1.40,1.20,3.61,1.92,8.50,12.55
1,SHUL4.SA,Auto Parts,24.46,25.8,13.20,14.0,15.6,19.4,0.4,3.18,5.26,7.8,1.01,1.02,1.64,1.42,8.15,7.32
2,CSUD3.SA,Software - Infrastructure,34.90,41.4,12.73,16.7,22.2,20.1,0.1,1.23,5.09,6.1,1.22,1.12,1.75,1.35,9.14,6.73
3,CSMG3.SA,Utilities - Regulated Water,39.42,43.4,14.47,17.8,16.1,17.8,0.7,1.73,5.68,11.9,1.01,1.08,0.89,1.08,7.36,6.05
4,KEPL3.SA,Farm & Heavy Construction Machinery,28.36,30.4,14.41,15.3,30.9,33.4,0.4,2.14,6.99,8.9,1.15,1.03,2.64,2.25,7.85,6.74
5,VULC3.SA,Footwear & Accessories,36.13,42.2,17.19,18.6,22.3,26.6,0.1,2.87,6.18,19.2,1.44,1.46,1.77,2.09,8.58,7.86
6,SAPR4.SA,Utilities - Regulated Water,59.62,58.3,21.69,22.1,13.0,14.2,0.6,1.75,5.36,5.0,1.27,1.25,0.83,0.80,5.44,5.63
9,PSSA3.SA,Insurance - Diversified,100.00,100.0,7.20,7.6,84.4,19.8,0.0,1.10,4.93,4.3,0.61,0.69,1.47,1.84,9.07,9.28
11,CPFE3.SA,Utilities - Regulated Electric,26.84,33.6,12.53,13.4,16.5,26.9,1.4,1.11,5.53,8.6,1.04,0.90,2.35,1.82,7.76,6.75
12,CMIG4.SA,Utilities - Diversified,20.66,21.8,11.61,20.7,13.9,28.5,0.5,1.04,7.36,11.1,0.70,0.85,1.13,1.17,5.96,4.10


In [ ]:
###############################################
### Filter Only Cheap Stocks - But not that BAD
###############################################

cheap_to_watch = data[

    (data['Mg. Bruta Med.'] >= 15)
    & ((data['Mg. Bruta Hj'] >= 15) | ((data['Setor'] == 'Intermediários Financeiros') & (data['Mg. Bruta Hj'] == 0)))

    & (data['Mg. Net Med'] > 1)
    & ((data['Mg. Net Hj'] > 1) | ((data['Setor'] == 'Intermediários Financeiros') & (data['Mg. Net Hj'] == 0)))

    & ((data['ROIC'] >= 1) | ((data['Setor'] == 'Intermediários Financeiros') & (data['ROIC'] == 0)))
    & (data['ROE'] >= 1)

    & (data['Div. Liq / Patr'] <= 2)
    & (
        (data['Liq Corrente'] >= 0.9)
        | ((data['Setor'] == 'Intermediários Financeiros') & (data['Liq Corrente'] == 0))
    )

    & (data['Div. Yield Med'] >= 2)
    & (data['Div. Yield Hj'] >= 2)
    & (data['Div. Yield Hj'] <= 20)

    & (data['PSR Med'] <= 4)
    & (data['PSR Hj'] <= data['PSR Med'] * 0.9)

    & (data['PVP Med'] <= 4)
    & (data['PVP Hj'] <= data['PVP Med'] * 0.9)

    & (data['PL Med'] > 0)
    & (data['PL Med'] <= 20)
    & (data['PL Hj'] > 0)
    & (data['PL Hj'] <= data['PL Med'] * 0.9)

  ].reset_index(drop=True)

cheap_to_watch = (
    cheap_to_watch[
        (~cheap_to_watch.ticker.isin(final_data.ticker))
        & (~cheap_to_watch.ticker.isin(to_watch.ticker))
    ]
    .reset_index(drop=True)
)

cheap_to_watch

,ticker,Setor,Mg. Bruta Med.,Mg. Bruta Hj,Mg. Net Med,Mg. Net Hj,ROIC,ROE,Div. Liq / Patr,Liq Corrente,Div. Yield Med,Div. Yield Hj,PSR Med,PSR Hj,PVP Med,PVP Hj,PL Med,PL Hj
0,MILS3.SA,Rental & Leasing Services,59.12,62.9,16.86,19.2,19.0,19.8,1.1,2.15,2.61,5.5,2.5,1.38,1.88,1.42,12.52,7.16


In [ ]:
###########################################
### FINAL STOCK PICKING - IT MUST BE MANUAL
###########################################

final_data = (
    data # LEVE3
    [(data.ticker == 'LEVE3.SA')]
    #._append(to_watch)
    #.append(cheap_to_watch[cheap_to_watch.ticker == 'ROMI3.SA'])
    .drop_duplicates(subset='ticker', keep="first")
    .reset_index(drop=True)
)

final_data

,ticker,Setor,Mg. Bruta Med.,Mg. Bruta Hj,Mg. Net Med,Mg. Net Hj,ROIC,ROE,Div. Liq / Patr,Liq Corrente,Div. Yield Med,Div. Yield Hj,PSR Med,PSR Hj,PVP Med,PVP Hj,PL Med,PL Hj
0,LEVE3.SA,Auto Parts,27.17,29.7,14.53,13.7,33.9,52.7,0.9,2.16,5.42,8.7,1.1,0.85,2.4,3.28,6.98,6.22


In [ ]:
### Save the output inside Google Drive
final_data.to_csv('/content/drive/My Drive/data_lake/stock_valuation.csv', encoding='utf-8', index=False)

In [ ]:
### Save the output inside Google Drive
int_data = pd.merge(df, data, on="ticker")

In [ ]:
int_data[
    (int_data.ticker == 'EGIE3.SA') |
    (int_data.ticker == 'LEVE3.SA') |
    (int_data.ticker == 'SHUL4.SA')][
        ['ticker', 'mkt',	'size',	'qld',	'momnt',	'liq',	'Mg. Bruta Med.',	'Mg. Bruta Hj',	'Mg. Net Med',	'Mg. Net Hj',	'Liq Corrente',	'Div. Yield Med',	'Div. Yield Hj',	'PSR Med',	'PSR Hj',	'PVP Med',	'PVP Hj',	'PL Med',	'PL Hj']
    ]

,ticker,mkt,size,qld,momnt,liq,Mg. Bruta Med.,Mg. Bruta Hj,Mg. Net Med,Mg. Net Hj,Liq Corrente,Div. Yield Med,Div. Yield Hj,PSR Med,PSR Hj,PVP Med,PVP Hj,PL Med,PL Hj
3,SHUL4.SA,0.467,0.217,0.059,0.005,0.109,24.46,25.8,13.20,14.0,3.18,5.26,7.8,1.01,1.02,1.64,1.42,8.15,7.32
37,EGIE3.SA,0.377,0.007,0.017,-0.026,0.013,47.04,55.2,22.59,39.0,1.02,6.36,6.8,2.76,2.86,4.06,2.83,12.21,7.34
53,LEVE3.SA,0.458,0.230,0.004,0.018,0.120,27.17,29.7,14.53,13.7,2.16,5.42,8.7,1.10,0.85,2.40,3.28,6.98,6.22
